In [49]:
!pip install transformers

In [50]:
!pip install hazm

In [52]:
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 349.5 kB/s eta 0:00:0000:0100:11
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 493.4 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 621.2 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 315.6 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 595.3 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 1.1 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 448.1 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 908.8 kB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 851.3 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5

In [1]:
import numpy as np
import hazm
import pandas as pd
from transformers import AutoTokenizer
import tensorflow as tf
import torch
import gc
import re
import torch
torch.cuda.empty_cache()
from __future__ import unicode_literals
from hazm import *

/home/server/anaconda3/envs/nlp/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-06-17 01:16:34.862018: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-17 01:16:34.897571: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-17 01:16:34.898697: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-17 01:16:35.607448: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
dff = pd.read_excel('pure.xlsx')
dff = dff.rename(columns={'متن': 'title', 'label':'polarity'})
# Define a function to map polarity values to aspect values
def map_polarity_to_aspect(polarity):
    if polarity == 1:
        return 'مثبت'
    elif polarity == 0:
        return 'خنثی'
    elif polarity == -1:
        return 'منفی'
    else:
        return None

# Add the 'aspect' column to the dataframe and set its values based on the 'polarity' column
dff['aspect'] = dff['polarity'].apply(map_polarity_to_aspect)

# Print the updated dataframe
dff

,بازه زمانی,توییت,توییت کننده,تاریخ توییت,نوع پیام,پاسخ,نقل‌قول,ناشر توییت,در تاریخ,title,pos,neg,polarity,aspect
0,انقلابی از 28 بهمن تا 4 اسفند,عجب ویروس هوشمندی است این #کرونا ! دقیقاً روز ...,علی‌اکبر رائفی پور⁦ 🇮🇷⁩,"01-12-1398, 0:24 PM",tweet,NaN,NaN,NaN,NaN,عجب ویروس هوشمندی است این #کرونا ! دقیقاً روز ...,NaN,NaN,0,خنثی
1,NaN,جنابان پیروز در انتخابات مجلس‌ شورا‌‌ #ما_مردم...,علی‌اکبر رائفی پور⁦ 🇮🇷⁩,"03-12-1398, 5:16 PM",tweet,NaN,NaN,NaN,NaN,جنابان پیروز در انتخابات مجلس‌ شورا‌‌ #ما_مردم...,1.0,NaN,1,مثبت
2,NaN,آنچه از مواضع پایداری در جلسه دیروز شنیدم، باو...,🇮🇷وحید یامین پور,"30-11-1398, 2:38 PM",tweet,NaN,NaN,NaN,NaN,آنچه از مواضع پایداری در جلسه دیروز شنیدم، باو...,NaN,NaN,0,خنثی
3,NaN,"ابتلاء من در این وانفسا، ""ترس"" بود. ترس از مطر...",🇮🇷وحید یامین پور,"03-12-1398, 2:26 PM",tweet,NaN,NaN,NaN,NaN,"ابتلاء من در این وانفسا، ""ترس"" بود. ترس از مطر...",NaN,NaN,0,خنثی
4,NaN,نرخ مشارکت از ساعتی ۱۲۰ هزار رای به ۱۴۰ هزار ر...,علی‌اکبر رائفی پور⁦ 🇮🇷⁩,"02-12-1398, 7:32 PM",tweet,NaN,NaN,NaN,NaN,نرخ مشارکت از ساعتی ۱۲۰ هزار رای به ۱۴۰ هزار ر...,NaN,NaN,0,خنثی
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8930,NaN,- استاد🙋‍♂️ + بله - چرا پنجم دی نظام چنج نشد؟ ...,حسین کـاشانی,"05-10-1398, 10:53 PM",Tweet,NaN,NaN,حسین کـاشانی,"05-10-1398, 10:53 PM",- استاد🙋‍♂️ + بله - چرا پنجم دی نظام چنج نشد؟ ...,NaN,NaN,0,خنثی
8931,NaN,بصیرت عمومی، دانش ملی، فهم و درک درست از مقتضی...,جواد کریمی قدوسی,"28-09-1398, 10:20 AM",Tweet,NaN,NaN,جواد کریمی قدوسی,"28-09-1398, 10:20 AM",بصیرت عمومی، دانش ملی، فهم و درک درست از مقتضی...,NaN,NaN,0,خنثی
8932,NaN,وصف العیش نصف العیش سید ابراهیم رئیسی :رئیس قو...,فئودور یاسروفسکی,"10-09-1398, 11:37 AM",Tweet,NaN,NaN,فئودور یاسروفسکی,"10-09-1398, 11:37 AM",وصف العیش نصف العیش سید ابراهیم رئیسی :رئیس قو...,NaN,NaN,0,خنثی
8933,NaN,فکر کنم دوباره آموزش و پرورش قراره بشه ستاد ان...,Mohammad Nosouhi محمدنصوحی,"17-09-1398, 2:45 PM",Tweet,NaN,NaN,Mohammad Nosouhi محمدنصوحی,"17-09-1398, 2:45 PM",فکر کنم دوباره آموزش و پرورش قراره بشه ستاد ان...,NaN,NaN,0,خنثی


In [94]:
dff['توییت'][2]


'آنچه از مواضع پایداری در جلسه دیروز شنیدم، باور نکردنی بود. آقای محصولی! به\u200cخاطر یک جمله که در حمایت از وحدت و آقای #قالیباف گفتم، اسم بنده را از لیست وحدت خط زدید! فقط برای مقابله با انحصارطلبی و باندبازی شما انصراف نمی\u200cدهم. رای من: #لیست_وحدت منهای سه نفر از سهمیه پایداری\xa0'

In [3]:
dff['title'] = dff['title'].replace('nan', np.nan)
dff['title'] = dff['title'].replace('Null', np.nan)
dff = dff.dropna(axis=0, subset=['title'])
dff['aspect'] = dff['aspect'].replace('nan', np.nan)
dff['aspect'] = dff['aspect'].replace('Null', np.nan)
dff = dff.dropna(axis=0, subset=['aspect'])

In [82]:
lst = dff['aspect'].unique()

In [83]:
lst

array(['خنثی', 'مثبت', 'منفی'], dtype=object)

In [5]:
numbers = [0, 1, 2]

In [6]:
ConceptScops_dictionary = dict(zip(lst, numbers))
ConceptScops_dictionary

{'خنثی': 0, 'مثبت': 1, 'منفی': 2}

In [7]:
ConceptScops_dictionary = dict(zip(lst, numbers))
inv_map = {v: k for k, v in ConceptScops_dictionary.items()}
dff["IndexOfConceptScops"] = np.nan
listofIndex = []
for i in dff['aspect']:
    listofIndex.append(ConceptScops_dictionary[i])
dff['IndexOfConceptScops'] = listofIndex
df = pd.DataFrame()
title_list =[]
ConceptScops_list = []
IndexOfConceptScops_list = []
title_list = dff["title"].to_numpy()
ConceptScops_list = dff["aspect"].to_numpy()
IndexOfConceptScops_list = dff["IndexOfConceptScops"].to_numpy()
df['sentence'] = title_list
df['aspect'] = ConceptScops_list
df['polarity'] = IndexOfConceptScops_list



df['aspect'] = df['aspect'].replace('nan', np.nan)
df = df.dropna(axis=0, subset=['aspect'])

df['sentence'] = df['sentence'].replace('nan', np.nan)
df = df.dropna(axis=0, subset=['sentence'])

df['polarity'] = df['polarity'].replace('nan', np.nan)
df = df.dropna(axis=0, subset=['polarity'])
df


,sentence,aspect,polarity
0,عجب ویروس هوشمندی است این #کرونا ! دقیقاً روز ...,خنثی,0
1,جنابان پیروز در انتخابات مجلس‌ شورا‌‌ #ما_مردم...,مثبت,1
2,آنچه از مواضع پایداری در جلسه دیروز شنیدم، باو...,خنثی,0
3,"ابتلاء من در این وانفسا، ""ترس"" بود. ترس از مطر...",خنثی,0
4,نرخ مشارکت از ساعتی ۱۲۰ هزار رای به ۱۴۰ هزار ر...,خنثی,0
...,...,...,...
8930,- استاد🙋‍♂️ + بله - چرا پنجم دی نظام چنج نشد؟ ...,خنثی,0
8931,بصیرت عمومی، دانش ملی، فهم و درک درست از مقتضی...,خنثی,0
8932,وصف العیش نصف العیش سید ابراهیم رئیسی :رئیس قو...,خنثی,0
8933,فکر کنم دوباره آموزش و پرورش قراره بشه ستاد ان...,خنثی,0


In [8]:
def CleanPersianText(text):
    _normalizer = hazm.Normalizer()
    text = _normalizer.normalize(text)
    # Clean the training data
    # First we lower case the text
    text = text.lower()
    # remove links
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', text)
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text)
    text = re.sub(
        r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''',
        '', text)
    text = re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0–9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0–9@:%_\+.~#?&//=]*)', '',
                text, flags=re.MULTILINE)  # to remove links that start with HTTP/HTTPS in the tweet
    text = re.sub(r'[-a-zA-Z0–9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0–9@:%_\+.~#?&//=]*)', '', text,
                flags=re.MULTILINE)  # to remove other url links
    # Remove usernames
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove punctuations
    text = re.sub(r'[^\w\s]', '', text)
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [9]:
df['sentence'] = df['sentence'].astype(str)
# Clean the text
df['sentence'] = df['sentence'].apply(CleanPersianText)

In [10]:
df['sentence']

0       عجب ویروس هوشمندی است این کرونا دقیقا روز قبل ...
1       جنابان پیروز در انتخابات مجلس شورا ما_مردم باب...
2       آنچه از مواضع پایداری در جلسه دیروز شنیدم باور...
3       ابتلاء من در این وانفسا ترس بود ترس از مطرود ش...
4       نرخ مشارکت از ساعتی هزار رای به هزار رای رسیده...
                              ...                        
8930    استاد بله چرا پنجم دی نظام چنج نشد چون مردم فر...
8931    بصیرت عمومی دانش ملی فهم و درک درست از مقتضیات...
8932    وصف العیش نصف العیش سید ابراهیم رئیسی رئیس قوه...
8933    فکر کنم دوباره آموزش و پرورش قراره بشه ستاد ان...
8934                   نامزدهای انتخابات شفافیت فساد مجلس
Name: sentence, Length: 8935, dtype: object

In [11]:
# Count the number of occurrences of each unique value in the "aspect" column
counts = df['aspect'].value_counts()

# Print the counts for each aspect
print("خنثی:", counts.get("خنثی", 0))
print("مثبت:", counts.get("مثبت", 0))
print("منفی:", counts.get("منفی", 0))

خنثی: 4842
مثبت: 1117
منفی: 2976


In [12]:
# Separate the dataframe into three separate dataframes
df1 = df[df['aspect'] == 'خنثی']
df2 = df[df['aspect'] == 'مثبت']
df3 = df[df['aspect'] == 'منفی']

In [13]:
df11 = df1.iloc[0:1116]
df12 = df1.iloc[1117:4841]
df2_new = df2.iloc[0:1116]
df31 = df3.iloc[0:1116]
df32 = df3.iloc[1117:2975]

In [14]:

# Concatenate the dataframes
df = pd.concat([df11, df2_new, df31])
df_test = pd.concat([df12, df32])
# Print the number of rows in the concatenated dataframe
print("Number of rows in the concatenated dataframe:", len(df))

Number of rows in the concatenated dataframe: 3348


In [15]:
# Remove the index of the dataframe
df = df.reset_index(drop=True)

In [16]:
df_test = df_test.reset_index(drop=True)

In [17]:
# Shuffle the concatenated dataframe
df = df.sample(frac=1, random_state=42)

df_test = df_test.sample(frac=1, random_state=42)
# Remove the index of the dataframe
df = df.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [18]:
df['polarity'].unique()

array([1, 2, 0])

In [19]:
df = df.rename(columns={'sentence': 'text', 'polarity': 'label'})

In [20]:
df = df.drop('aspect', axis=1)

In [21]:
df

,text,label
0,مسئولین امنیتی عزیز حواستون به برخی از رد صلاح...,1
1,روش گزارشدهی رسانههای بزرگ آمریکا از انتخابات ...,2
2,سیروان خسروی گفته در انتخابات مهراب قاسمخانی ا...,1
3,خدایا مگه لیست اصلاحات کرونا داشت کسی بهشون را...,1
4,امپراتوری دروغ و رسوایی پشت رسوایی مهمان آمریک...,2
...,...,...
3343,اخیرا بیانیهای با عکس و اسم جمعی از چهرههای اص...,0
3344,احمد مازنی رئیس کمیسیون فرهنگی مجلس همان اصلاح...,1
3345,تبریک بالاخره قالیباف انتخابات بی رقیب را برد ...,1
3346,هر کسی بره پای صندوق رای از نظر من هیچ فرقی با...,0


In [22]:
import torch.nn as nn

tweet= df['text']
Label = df['label']

# Load the pre-trained BERT model and tokenizer
import torch
from transformers import BertModel, BertTokenizer
from transformers import BertTokenizer, BertForSequenceClassification
# tokenizer = BertTokenizer.from_pretrained('HooshvareLab/bert-fa-base-uncased-clf-persiannews')
# model = BertModel.from_pretrained('HooshvareLab/bert-fa-base-uncased-clf-persiannews')
tokenizer = BertTokenizer.from_pretrained('./model')
model = BertModel.from_pretrained('./model')

# Create a linear layer to project the BERT vectors to the desired length
projection_layer = nn.Linear(768, 100)

# Convert the tweets into BERT vectors
def bert_vector(text):
    encoded_input = tokenizer(text, padding=True, truncation=True, max_length=128, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    last_hidden_state = model_output.last_hidden_state
    mean_pooled_output = torch.mean(last_hidden_state, dim=1).squeeze()

    # Apply the projection layer to reduce the vector length to 100
    reduced_vector = projection_layer(mean_pooled_output)

    return list(reduced_vector.detach().numpy())

Some weights of the model checkpoint at ./model were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [23]:
import numpy as np
import torch

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import BertForSequenceClassification, BertTokenizer, TrainingArguments, Trainer

In [24]:

# Load BERT model and tokenizer
model_name = './model'
tokenizer = BertTokenizer.from_pretrained(model_name,ignore_mismatched_sizes=True)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3,ignore_mismatched_sizes=True)  # Change num_labels to the number of classes you have


In [25]:
df['label'].value_counts()

1    1116
2    1116
0    1116
Name: label, dtype: int64

In [26]:
df['label'] = df['label'].apply(lambda x : int(x))

In [27]:

# Prepare dataset
def encode(sentences, labels):
    encoded = tokenizer(sentences, truncation=True, padding=True, return_tensors="pt")
    return {"input_ids": encoded["input_ids"], "attention_mask": encoded["attention_mask"], "labels": torch.tensor(labels)}

# Replace with your own data
# sentences = ["This is a positive sentence", "This is a negative sentence"]
# labels = [1, 0]

train_sentences, val_sentences, train_labels, val_labels = train_test_split(df['text'].tolist(),df['label'].tolist() , test_size=0.2, random_state=42)




In [28]:
from transformers import AutoTokenizer
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, sentences, labels, tokenizer, max_length):
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(sentence, truncation=True, max_length=self.max_length, padding='max_length', return_tensors='pt')
        # Remove the batch dimension from input tensors
        for key in encoding:
            encoding[key] = encoding[key].squeeze(0)
        encoding['labels'] = torch.tensor(label, dtype=torch.long)
        return encoding

    def select(self, indices):
        new_sentences = [self.sentences[i] for i in indices]
        new_labels = [self.labels[i] for i in indices]
        return CustomDataset(new_sentences, new_labels, self.tokenizer, self.max_length)

# tokenizer = AutoTokenizer.from_pretrained('MODEL_NAME')  # Replace 'MODEL_NAME' with the name or path of the model you are using
MAX_LENGTH = 128  # Adjust the maximum length of the tokenized sequences based on your model and task

train_dataset = CustomDataset(train_sentences, train_labels, tokenizer, MAX_LENGTH)
val_dataset = CustomDataset(val_sentences, val_labels, tokenizer, MAX_LENGTH)


In [29]:
import torch
def gpu_check():
    torch.__version__

    # If there's a GPU available...
    if torch.cuda.is_available():    
        # Tell PyTorch to use the GPU.    
        device = torch.device("cuda")

        print('There are %d GPU(s) available.' % torch.cuda.device_count())

        print('We will use the GPU:', torch.cuda.get_device_name(0))

    # If not...
    else:
        print('No GPU available, using the CPU instead.')
        device = torch.device("cpu")
        
    return device
gpu_check()

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 2080 Ti


device(type='cuda')

In [30]:

# Define accuracy metric
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Set training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=6,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    logging_dir="./logs",
    learning_rate=5e-5,
)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# Train and evaluate model
trainer.train()
trainer.evaluate()

/home/server/anaconda3/envs/nlp/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2678
  Num Epochs = 6
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1008
 17%|█▋        | 168/1008 [00:28<02:07,  6.57it/s]***** Running Evaluation *****
  Num examples = 670
  Batch size = 16
                                                  
 17%|█▋        | 169/1008 [00:31<11:59,  1.17it/s]

{'eval_loss': 0.9267472624778748, 'eval_accuracy': 0.5313432835820896, 'eval_runtime': 2.3217, 'eval_samples_per_second': 288.582, 'eval_steps_per_second': 18.09, 'epoch': 1.0}


 33%|███▎      | 336/1008 [00:59<01:40,  6.66it/s]***** Running Evaluation *****
  Num examples = 670
  Batch size = 16
                                                  
 33%|███▎      | 337/1008 [01:02<09:31,  1.17it/s]

{'eval_loss': 1.0009666681289673, 'eval_accuracy': 0.5701492537313433, 'eval_runtime': 2.3017, 'eval_samples_per_second': 291.087, 'eval_steps_per_second': 18.247, 'epoch': 2.0}


 50%|████▉     | 500/1008 [01:29<01:21,  6.22it/s]Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json


...

{'loss': 0.0466, 'learning_rate': 3.9682539682539683e-07, 'epoch': 5.95}


Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
100%|██████████| 1008/1008 [03:49<00:00,  1.36it/s]***** Running Evaluation *****
  Num examples = 670
  Batch size = 16
                                                   
100%|██████████| 1008/1008 [03:51<00:00,  1.36it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 1008/1008 [03:51<00:00,  4.36it/s]
***** Running Evaluation *****
  Num examples = 670
  Batch size = 16


{'eval_loss': 2.4963629245758057, 'eval_accuracy': 0.591044776119403, 'eval_runtime': 2.3204, 'eval_samples_per_second': 288.746, 'eval_steps_per_second': 18.101, 'epoch': 6.0}
{'train_runtime': 231.4674, 'train_samples_per_second': 69.418, 'train_steps_per_second': 4.355, 'train_loss': 0.3467430027163891, 'epoch': 6.0}


100%|██████████| 42/42 [00:02<00:00, 18.68it/s]


{'eval_loss': 2.4963629245758057,
 'eval_accuracy': 0.591044776119403,
 'eval_runtime': 2.324,
 'eval_samples_per_second': 288.298,
 'eval_steps_per_second': 18.072,
 'epoch': 6.0}

In [203]:

import pandas as pd

# Read the CSV file into a DataFrame
new_data = pd.read_csv('twt_siasi_final_4.csv', delimiter = '|')
# new_data = dff.head(5)
# Print the DataFrame




/tmp/ipykernel_1100312/3263100654.py:4: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  new_data = pd.read_csv('twt_siasi_final_4.csv', delimiter = '|')


In [204]:
new_data = new_data[:5000]

In [205]:
new_data


,Unnamed: 0,politicalTag,FK_LanguageId_Status,opinion,mainTag,Type,cleanText_Status,Created_Status,ReplyCount_Status,QuoteCount_Status,HashCText_Status,pK_StatusId,Text_Status,FK_UserIDBytype234,Lang_Status,RetweetCount_Status,FavoriteCount_Status,TypeName,fK_InReplyToStatusID,fK_QuotedStatusID,fK_RetweetStatusID,cleanText_Tweet,CreatedAtTweet,ReplyCount_Tweet,fK_LanguageId_Tweet,QuoteCount_Tweet,PK_StatusId,Text_Tweet,Fk_UserCreatorTweet,lang_Tweet,RetweetCount_Tweet,FavoriteCount_Tweet,user_id,Verified_Status,UserName_Status,ScreenName_Status,Verified_CreatorTweet,UserName_CreatorTweet,ScreenName_CreatorTweet,Took,Total,MediaURL,ExpandedUrl,ProfileImageUrl_Status,ProfileImageUrl_CreatorTweet,fK_UserId,ViewCount_Status,message_weight,ViewCount_Tweet,ProfileImageCreator,fK_InReplyToUserID,aspect_sentiment,emotion,mod_label
0,0,10050000,102097,-1,8000000,2,سانازجان این وحشیگریها44سالست ادامه دارد بمددت...,2023-03-02 14:10:38.000,0,0,8636549831242736640,1631243105893511168,سانازجان این وحشیگریها44سالست ادامه دارد\nبمدد...,772457957829316608,fa,0,0,NaN,0,0,1631196831328661504,سانازجان این وحشیگریها44سالست ادامه دارد بمددت...,2023-03-02 11:06:45.000,0,102097,0,1631196831328661504,سانازجان این وحشیگریها44سالست ادامه دارد\nبمدد...,1568939256818667520,fa,2,2,772457957829316608,False,Mohammad Foradi,ForadiMohammad,0.0,👑شیخ مفید👑,MfydShykh,2459.0,323062.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'aspect': 'امریکا', 'sentiment': 'منفی'}, {'...",delighted - خیلی خوشحال,دشمن
1,1,-100,102097,-1,-100,2,جنگ همین است. یعنی در کشوری که ظرف چهل سال نرخ...,2023-03-02 14:08:42.000,0,0,8981787607175172096,1631242620591652864,«جنگ» همین است. یعنی در کشوری که ظرف چهل سال، ...,1628632709756080128,fa,0,0,NaN,0,0,1631214500522123264,جنگ همین است. یعنی در کشوری که ظرف چهل سال نرخ...,2023-03-02 12:16:58.000,24,102097,9,1631214500522123264,«جنگ» همین است. یعنی در کشوری که ظرف چهل سال، ...,747320835728277504,fa,198,765,1628632709756080128,False,سید محمد حسینی موسوی🇮🇷,Mousavei1991,0.0,⛰️Παρνασσός⁦Z🇵🇸,Parnassus1871,NaN,NaN,NaN,NaN,NaN,bucketId=user&objectId=747320835728277504/d075...,7.473208e+17,NaN,NaN,NaN,NaN,NaN,"[{'aspect': 'سال', 'sentiment': 'منفی'}, {'asp...",neutral - خنثی,0
2,2,10070000,102097,-1,8000000,2,نگران نباش دریای جنازه تو غوطه شرقی به کمک قاس...,2023-03-02 14:03:53.000,0,0,5696789287847169024,1631241408668479488,نگران نباش، دریای جنازه تو غوطه شرقی به کمک قا...,1433893953124896768,fa,0,0,NaN,0,1630918667910238208,1630964915199508480,نگران نباش دریای جنازه تو غوطه شرقی به کمک قاس...,2023-03-01 19:45:12.000,4,102097,0,1630964915199508480,نگران نباش، دریای جنازه تو غوطه شرقی به کمک قا...,1512793624744009728,fa,61,487,1433893953124896768,False,KaAva,AvayeTo,0.0,دکتر ملاتاریا,Mollataria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90363.0,NaN,NaN,"[{'aspect': 'امپریالیستی', 'sentiment': 'مثبت'...",neutral - خنثی,اصلاح طلب
3,3,-100,102097,-1,-100,2,در طول تاریخ تنها کشوری که از تمامیت ارضی ایرا...,2023-03-02 14:03:24.000,0,0,3998322769218584064,1631241289101455360,در طول تاریخ، تنها کشوری که از تمامیت ارضی ایر...,1598071245333037056,fa,0,0,NaN,0,0,1631038006436982784,در طول تاریخ تنها کشوری که از تمامیت ارضی ایرا...,2023-03-02 00:35:38.000,21,102097,4,1631038006436982784,در طول تاریخ، تنها کشوری که از تمامیت ارضی ایر...,1509613366674923520,fa,140,1863,1598071245333037056,False,Free Iran,freedom4iran28,0.0,🏴Jannatkhah.ir 🔩 🪰,Jannatkhah_ir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'aspect': 'امپریالیسم', 'sentiment': 'مثبت'}...",neutral - خنثی,سلطنت طلب
4,4,10150000,102097,-1,7000000,2,جنگ همین است. یعنی در کشوری که ظرف چهل سال نرخ...,2023-03-02 14:01:33.000,0,0,8981787607175172096,1631240819603546112,«جنگ» همین است. یعنی در کشوری که ظرف چهل سال، ...,332568495,fa,0,0,NaN,0,0,1631214500522123264,جنگ همین است. یعنی در کشوری که ظرف چهل سال نرخ...,2023-03-02 12:16:58.000,24,102097,9,1631214500522123264,«جنگ» همین است. یعنی در کشوری که ظرف چهل سال، ...,747320835728277504,fa,198,765,332568495,False,امید لک,OmidlMazaheri,0.0,⛰️Παρνασσός⁦Z🇵🇸,Parnassus1

In [206]:
from transformers import BertForSequenceClassification, BertTokenizer
# model_path = "./results/checkpoint-1000"
model = BertForSequenceClassification.from_pretrained("./results")
tokenizer = BertTokenizer.from_pretrained("./model")


loading configuration file ./results/config.json
Model config BertConfig {
  "_name_or_path": "./model",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.22.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 100000
}

loading weights file ./results/pytorch_model.bin
All model checkpoint weights were used w

In [207]:
import torch.nn.functional as F
from transformers import BertForSequenceClassification, BertTokenizer
# model_path = "./results/checkpoint-1000"
model = BertForSequenceClassification.from_pretrained("./results/checkpoint-1000")
tokenizer = BertTokenizer.from_pretrained("./model")

# Label_mapping = {0: 'خنثی',1: 'مثبت',2: 'منفی'}
# Label_mapping = {0: '0',1: '1',2: '2'}
predictions = []

# new_data['predict_cleanText_status'] = ''

for index, row in new_data.iterrows():
    text = row['cleanText_Status']
    text_cleaned = CleanPersianText(text)
    encoded_input = tokenizer(text_cleaned, padding=True, truncation=True, max_length= 128, return_tensors="pt")
    with torch.no_grad():
        model_output = model(**encoded_input)
        logits = model_output.logits
        softmax_tensor = F.softmax(logits, dim =1)
        max_value,max_index = torch.max(softmax_tensor,dim =1)
        predictions.append(max_value.item())
        # print(predictions)


        # max_value,max_index = torch.max(logits,dim =1)
        # print(max_value)
        # max_index = torch.argmax(logits)
        # label = Label_mapping[max_index.item()]
        # predictions.append(max_value.item())
        # print(predictions)

new_data['hope_predict'] = predictions
new_data.to_csv('hope-ofool_prediction3.csv', index = False)


loading configuration file ./results/checkpoint-1000/config.json
Model config BertConfig {
  "_name_or_path": "./model",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.22.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 100000
}

loading weights file ./results/checkpoint-1000/pytorch_model.bin
All mode

In [143]:
print(new_data['cleanText_Status'][3])

در طول تاریخ تنها کشوری که از تمامیت ارضی ایران دفاع کرده امپریالیسم امریکا بوده اخراج شوروی از آذربایجان در سال 1325 تنها کشوری که با کشورگشایی هر زمان که تونسته از ایران و کشورهای همسایه اش خاک گرفته امپریالیسم روسیه بوده چپ ضد امپریالیسم مدل خاردار روسی رو دوست دارن


In [161]:
pd.set_option('display.max_columns', 500)


In [164]:
new_data.head(100)

,Unnamed: 0,politicalTag,FK_LanguageId_Status,opinion,mainTag,Type,cleanText_Status,Created_Status,ReplyCount_Status,QuoteCount_Status,HashCText_Status,pK_StatusId,Text_Status,FK_UserIDBytype234,Lang_Status,RetweetCount_Status,FavoriteCount_Status,TypeName,fK_InReplyToStatusID,fK_QuotedStatusID,fK_RetweetStatusID,cleanText_Tweet,CreatedAtTweet,ReplyCount_Tweet,fK_LanguageId_Tweet,QuoteCount_Tweet,PK_StatusId,Text_Tweet,Fk_UserCreatorTweet,lang_Tweet,RetweetCount_Tweet,FavoriteCount_Tweet,user_id,Verified_Status,UserName_Status,ScreenName_Status,Verified_CreatorTweet,UserName_CreatorTweet,ScreenName_CreatorTweet,Took,Total,MediaURL,ExpandedUrl,ProfileImageUrl_Status,ProfileImageUrl_CreatorTweet,fK_UserId,ViewCount_Status,message_weight,ViewCount_Tweet,ProfileImageCreator,fK_InReplyToUserID,aspect_sentiment,emotion,mod_label,hope_predict
0,0,10050000,102097,-1,8000000,2,سانازجان این وحشیگریها44سالست ادامه دارد بمددت...,2023-03-02 14:10:38.000,0,0,8636549831242736640,1631243105893511168,سانازجان این وحشیگریها44سالست ادامه دارد\nبمدد...,772457957829316608,fa,0,0,NaN,0,0,1631196831328661504,سانازجان این وحشیگریها44سالست ادامه دارد بمددت...,2023-03-02 11:06:45.000,0,102097,0,1631196831328661504,سانازجان این وحشیگریها44سالست ادامه دارد\nبمدد...,1568939256818667520,fa,2,2,772457957829316608,False,Mohammad Foradi,ForadiMohammad,0.0,👑شیخ مفید👑,MfydShykh,2459.0,323062.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'aspect': 'امریکا', 'sentiment': 'منفی'}, {'...",delighted - خیلی خوشحال,دشمن,0
1,1,-100,102097,-1,-100,2,جنگ همین است. یعنی در کشوری که ظرف چهل سال نرخ...,2023-03-02 14:08:42.000,0,0,8981787607175172096,1631242620591652864,«جنگ» همین است. یعنی در کشوری که ظرف چهل سال، ...,1628632709756080128,fa,0,0,NaN,0,0,1631214500522123264,جنگ همین است. یعنی در کشوری که ظرف چهل سال نرخ...,2023-03-02 12:16:58.000,24,102097,9,1631214500522123264,«جنگ» همین است. یعنی در کشوری که ظرف چهل سال، ...,747320835728277504,fa,198,765,1628632709756080128,False,سید محمد حسینی موسوی🇮🇷,Mousavei1991,0.0,⛰️Παρνασσός⁦Z🇵🇸,Parnassus1871,NaN,NaN,NaN,NaN,NaN,bucketId=user&objectId=747320835728277504/d075...,7.473208e+17,NaN,NaN,NaN,NaN,NaN,"[{'aspect': 'سال', 'sentiment': 'منفی'}, {'asp...",neutral - خنثی,0,0
2,2,10070000,102097,-1,8000000,2,نگران نباش دریای جنازه تو غوطه شرقی به کمک قاس...,2023-03-02 14:03:53.000,0,0,5696789287847169024,1631241408668479488,نگران نباش، دریای جنازه تو غوطه شرقی به کمک قا...,1433893953124896768,fa,0,0,NaN,0,1630918667910238208,1630964915199508480,نگران نباش دریای جنازه تو غوطه شرقی به کمک قاس...,2023-03-01 19:45:12.000,4,102097,0,1630964915199508480,نگران نباش، دریای جنازه تو غوطه شرقی به کمک قا...,1512793624744009728,fa,61,487,1433893953124896768,False,KaAva,AvayeTo,0.0,دکتر ملاتاریا,Mollataria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90363.0,NaN,NaN,"[{'aspect': 'امپریالیستی', 'sentiment': 'مثبت'...",neutral - خنثی,اصلاح طلب,1
3,3,-100,102097,-1,-100,2,در طول تاریخ تنها کشوری که از تمامیت ارضی ایرا...,2023-03-02 14:03:24.000,0,0,3998322769218584064,1631241289101455360,در طول تاریخ، تنها کشوری که از تمامیت ارضی ایر...,1598071245333037056,fa,0,0,NaN,0,0,1631038006436982784,در طول تاریخ تنها کشوری که از تمامیت ارضی ایرا...,2023-03-02 00:35:38.000,21,102097,4,1631038006436982784,در طول تاریخ، تنها کشوری که از تمامیت ارضی ایر...,1509613366674923520,fa,140,1863,1598071245333037056,False,Free Iran,freedom4iran28,0.0,🏴Jannatkhah.ir 🔩 🪰,Jannatkhah_ir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'aspect': 'امپریالیسم', 'sentiment': 'مثبت'}...",neutral - خنثی,سلطنت طلب,0
4,4,10150000,102097,-1,7000000,2,جنگ همین است. یعنی در کشوری که ظرف چهل سال نرخ...,2023-03-02 14:01:33.000,0,0,8981787607175172096,1631240819603546112,«جنگ» همین است. یعنی در کشوری که ظرف چهل سال، ...,332568495,fa,0,0,NaN,0,0,1631214500522123264,جنگ همین است. یعنی در کشوری که ظرف چهل سال نرخ...,2023-03-02 12:16:58.000,24,102097,9,1631214500522123264,«جنگ» همین است. یعنی در کشوری که ظرف چهل سال، ...,747320835728277504,fa,198,765,332568495,False,امید لک,OmidlMazaheri,0.0,⛰️Παρ

In [156]:
print(new_data['cleanText_Status'][762626])

دشمنی آمریکا و بهانه گیریهای آنها تمام شدنی نیست. این دشمنی دستاویزی برای نابودی تفکر سیاست و زیاده خواهی دولت آمریکا و نهایتا دگرگونی این رژیم منحوس خواهد شد #حقوق_بشر_آمریکایی
